In [1]:
import pyaudio
import matplotlib.mlab as mlab
import numpy as np
import tensorflow as tf
import time

from queue import Queue
from threading import Thread

/Users/az01640/Projets/multrigger-word/.venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/az01640/Projets/multrigger-word/.venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/az01640/Projets/multrigger-word/.venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/az01640/P

In [2]:
import sys
import os

sys.path.append("/".join(os.getcwd().split("/")[:-1]))

In [3]:
from src.settings.general import FRAME_RATE

In [4]:
from src.settings.trigger import TX, FX, N_CLASSES, TRIGGER_KERNEL_SIZE, TRIGGER_STRIDE, SAMPLE_DURATION_MS

In [5]:
from src.utils.audio import load_raw_audio

In [6]:
from src.utils.audio import get_spectrogram
from src.trigger.make_dataset import transform_labels

In [8]:
data = None
silence_threshold = 100
RECORD_SECONDS = 5

q = Queue()

CHUNK_DURATION = FRAME_RATE * (SAMPLE_DURATION_MS / 1000)

def callback(in_data, frame_count, time_info, status):
    global data, silence_threshold

    delta_data = np.frombuffer(in_data, dtype='int16')
    sys.stdout.write('.')
    
    if np.abs(delta_data).mean() < silence_threshold:
        #sys.stdout.write('-')
        return (in_data, pyaudio.paContinue)
    #else:
        #sys.stdout.write('.')

    data = np.append(data,delta_data)

    if len(data) > CHUNK_DURATION:
        data = data[-CHUNK_DURATION:]
        # Process data async by sending a queue.
        q.put(data)

    return (in_data, pyaudio.paContinue)

def get_stream_from_mic(audio, callback):
    stream = audio.open(format=pyaudio.paInt16,
                        channels=1,
                        rate=FRAME_RATE,
                        input=True,
                        frames_per_buffer=4096,
                        input_device_index=0,
                        stream_callback=callback)
    return stream

audio = pyaudio.PyAudio()
stream = get_stream_from_mic(audio, callback)
stream.start_stream()

frames = []
 
for i in range(0, int(FRAME_RATE / 4096 * RECORD_SECONDS)):
    data = q.get()

# stop Recording
stream.stop_stream()
stream.close()
audio.terminate()



........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

KeyboardInterrupt: 

In [2]:
import pyaudio
import wave
 
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
CHUNK = 1024
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "file.wav"
 
audio = pyaudio.PyAudio()
 
# start Recording
stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True,
                frames_per_buffer=CHUNK)

frames = []
 
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)


# stop Recording
stream.stop_stream()
stream.close()
audio.terminate()
 
waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
waveFile.setnchannels(CHANNELS)
waveFile.setsampwidth(audio.get_sample_size(FORMAT))
waveFile.setframerate(RATE)
waveFile.writeframes(b''.join(frames))
waveFile.close()

In [13]:
stream.stop_stream()
stream.close()